## Analytics: Cubes and Rollups

or
## Buying booties for beasts

### Setup

In [2]:
!pip install ipython_sql

You are using pip version 7.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
!pip install psycopg2

  Using cached psycopg2-2.6.2.tar.gz
  Running setup.py install for psycopg2
You are using pip version 7.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [21]:
!createdb animals

createdb: database creation failed: ERROR:  database "animals" already exists


In [22]:
!psql -f setup.sql animals

psql:setup.sql:1: NOTICE:  drop cascades to constraint species_class_id_fkey on table species
DROP TABLE
CREATE TABLE
INSERT 0 3
psql:setup.sql:9: NOTICE:  drop cascades to constraint animal_species_id_fkey on table animal
DROP TABLE
CREATE TABLE
INSERT 0 5
DROP TABLE
CREATE TABLE
INSERT 0 8


In [4]:
%load_ext sql

In [7]:
%sql postgresql://:@/animals

'Connected: @animals'

### Data

In [23]:
%%sql
SELECT a.*,
       s.name AS species
FROM   animal a
JOIN   species s ON (a.species_id = s.id)

8 rows affected.


name,species_id,limbs,color,tame,species
Bumpy,1,4,black,True,cat
Rascal,1,3,orange,True,cat
Noah,1,4,gray,False,cat
Biting,3,4,bay,True,horse
Prince,3,4,gray,True,horse
None,5,10,None,False,cuttlefish
None,4,4,green,False,lizard
Tachyon,2,4,green,False,turtle


In [10]:
%%sql
SELECT SUM(a.limbs), 
       c.name AS class
FROM   animal a
JOIN   species s ON (a.species_id = s.id)
JOIN   bio_class c ON (s.class_id = c.id)
GROUP BY c.name


3 rows affected.


sum,class
10,mollusk
8,reptile
19,mammal


In [24]:
%%sql
SELECT SUM(a.limbs), 
       a.color,
       c.name AS class
FROM   animal a
JOIN   species s ON (a.species_id = s.id)
JOIN   bio_class c ON (s.class_id = c.id)
GROUP BY (a.color, c.name)

6 rows affected.


sum,color,class
4,bay,mammal
4,black,mammal
8,gray,mammal
10,None,mollusk
3,orange,mammal
8,green,reptile


In [25]:
%%sql
SELECT SUM(a.limbs), 
       a.tame,
       c.name AS class
FROM   animal a
JOIN   species s ON (a.species_id = s.id)
JOIN   bio_class c ON (s.class_id = c.id)
GROUP BY (a.tame, c.name)

4 rows affected.


sum,tame,class
10,False,mollusk
4,False,mammal
8,False,reptile
15,True,mammal


So far so good.  But what if 

- the table is huge
- we need to precalculate values
- user may need variety of groupings
  - species
  - class
  - color
  - tame
  - species + color
  - color + tame
  - class + color + tame
  - ...

### Cubes

In [28]:
%%sql
SELECT SUM(a.limbs), 
       a.color,`
       a.tame,
       s.name AS species,
       c.name AS class
FROM   animal a
JOIN   species s ON (a.species_id = s.id)
JOIN   bio_class c ON (s.class_id = c.id)
GROUP BY CUBE (color, tame, species, class)
ORDER BY (a.color, a.tame, s.name, c.name) DESC NULLS FIRST;

90 rows affected.


sum,color,tame,species,class
37,None,None,None,None
10,None,None,None,None
8,None,None,None,reptile
10,None,None,None,mollusk
10,None,None,None,mollusk
19,None,None,None,mammal
4,None,None,turtle,None
4,None,None,turtle,reptile
4,None,None,lizard,None
4,None,None,lizard,reptile


In [30]:
%%sql
SELECT * FROM (
    SELECT SUM(a.limbs), 
           a.color,
           a.tame,
           s.name AS species,
           c.name AS class
    FROM   animal a
    JOIN   species s ON (a.species_id = s.id)
    JOIN   bio_class c ON (s.class_id = c.id)
    GROUP BY CUBE (color, tame, species, class)
    ORDER BY (a.color, a.tame, s.name, c.name) DESC NULLS FIRST
    ) subcube 
WHERE color = 'gray' AND tame = true AND species = 'horse' AND class = 'mammal';

1 rows affected.


sum,color,tame,species,class
4,gray,True,horse,mammal


All my tame horse mammals, any color (the wrong way)

In [31]:
%%sql
SELECT * FROM (
    SELECT SUM(a.limbs), 
           a.color,
           a.tame,
           s.name AS species,
           c.name AS class
    FROM   animal a
    JOIN   species s ON (a.species_id = s.id)
    JOIN   bio_class c ON (s.class_id = c.id)
    GROUP BY CUBE (color, tame, species, class)
    ORDER BY (a.color, a.tame, s.name, c.name) DESC NULLS FIRST
    ) subcube 
WHERE color IS NULL AND tame = true AND species = 'horse' AND class = 'mammal';

1 rows affected.


sum,color,tame,species,class
8,None,True,horse,mammal


Why is that the wrong way?

Well, let's try buying booties for cuttlefish.

In [34]:
%%sql
SELECT * FROM (
    SELECT SUM(a.limbs), 
           a.color,
           a.tame,
           s.name AS species,
           c.name AS class
    FROM   animal a
    JOIN   species s ON (a.species_id = s.id)
    JOIN   bio_class c ON (s.class_id = c.id)
    GROUP BY CUBE (color, tame, species, class)
    ORDER BY (a.color, a.tame, s.name, c.name) DESC NULLS FIRST
    ) subcube 
WHERE color IS NULL AND tame = false AND species = 'cuttlefish' AND class = 'mollusk';

2 rows affected.


sum,color,tame,species,class
10,None,False,cuttlefish,mollusk
10,None,False,cuttlefish,mollusk


One row is `color=NULL` because it represents all cuttlefish colors.

One row is `color=NULL` because it represents cuttlefish whose color is  `NULL`.

The proper way to distingush these rows is with `GROUPING`.

`GROUPING` forms a bitmap; for each position,

- `0` if the row represents a specific column value
- `1` if the row represents all values of that column

In [35]:
%%sql
SELECT sc.*, sc.grouping_bitmap::bit(4) FROM (
    SELECT SUM(a.limbs), 
           a.color,
           a.tame,
           s.name AS species,
           c.name AS class,
           GROUPING(a.color, a.tame, s.name, c.name) AS grouping_bitmap
    FROM   animal a
    JOIN   species s ON (a.species_id = s.id)
    JOIN   bio_class c ON (s.class_id = c.id)
    GROUP BY CUBE (color, tame, species, class)
    ORDER BY (a.color, a.tame, s.name, c.name) DESC NULLS FIRST
    ) sc 

90 rows affected.


sum,color,tame,species,class,grouping_bitmap,grouping_bitmap_1
37,None,None,None,None,15,1111
10,None,None,None,None,7,0111
8,None,None,None,reptile,14,1110
10,None,None,None,mollusk,14,1110
10,None,None,None,mollusk,6,0110
19,None,None,None,mammal,14,1110
4,None,None,turtle,None,13,1101
4,None,None,turtle,reptile,12,1100
4,None,None,lizard,None,13,1101
4,None,None,lizard,reptile,12,1100


In [40]:
%%sql
SELECT sc.*, sc.grouping_bitmap::bit(4) FROM (
    SELECT SUM(a.limbs), 
           a.color,
           a.tame,
           s.name AS species,
           c.name AS class,
           GROUPING(a.color, a.tame, s.name, c.name) AS grouping_bitmap
    FROM   animal a
    JOIN   species s ON (a.species_id = s.id)
    JOIN   bio_class c ON (s.class_id = c.id)
    GROUP BY CUBE (color, tame, species, class)
    ORDER BY (a.color, a.tame, s.name, c.name) DESC NULLS FIRST
    ) sc 
WHERE grouping_bitmap & 8 = 8
AND   tame = false AND species = 'cuttlefish' AND class = 'mollusk';

1 rows affected.


sum,color,tame,species,class,grouping_bitmap,grouping_bitmap_1
10,None,False,cuttlefish,mollusk,8,1000


Species and class have a hierarchical relationship.  (No non-mollusk cuttlefish)

In [48]:
%%sql
SELECT sc.*, sc.grouping_bitmap::bit(2) FROM (
    SELECT SUM(a.limbs), 
           s.name AS species,
           c.name AS class,
           GROUPING(s.name, c.name) AS grouping_bitmap
    FROM   animal a
    JOIN   species s ON (a.species_id = s.id)
    JOIN   bio_class c ON (s.class_id = c.id)
    GROUP BY CUBE (species, class)
    ORDER BY (s.name, c.name) DESC NULLS FIRST
    ) sc ;

14 rows affected.


sum,species,class,grouping_bitmap,grouping_bitmap_1
37,None,None,3,11
8,None,reptile,2,10
10,None,mollusk,2,10
19,None,mammal,2,10
4,turtle,None,1,01
4,turtle,reptile,0,00
4,lizard,None,1,01
4,lizard,reptile,0,00
8,horse,None,1,01
8,horse,mammal,0,00


In [49]:
%%sql
SELECT sc.*, sc.grouping_bitmap::bit(2) FROM (
    SELECT SUM(a.limbs), 
           s.name AS species,
           c.name AS class,
           GROUPING(c.name, s.name) AS grouping_bitmap
    FROM   animal a
    JOIN   species s ON (a.species_id = s.id)
    JOIN   bio_class c ON (s.class_id = c.id)
    GROUP BY ROLLUP (class, species)
    ORDER BY (c.name, s.name) DESC NULLS FIRST
    ) sc ;

9 rows affected.


sum,species,class,grouping_bitmap,grouping_bitmap_1
37,None,None,3,11
8,None,reptile,1,01
4,turtle,reptile,0,00
4,lizard,reptile,0,00
10,None,mollusk,1,01
10,cuttlefish,mollusk,0,00
19,None,mammal,1,01
8,horse,mammal,0,00
11,cat,mammal,0,00


Multiple forms of grouping can be combined

In [51]:
%%sql
SELECT sc.*, sc.grouping_bitmap::bit(4) FROM (
    SELECT SUM(a.limbs), 
           a.color,
           a.tame,
           s.name AS species,
           c.name AS class,
           GROUPING(a.color, a.tame, c.name, s.name) AS grouping_bitmap
    FROM   animal a
    JOIN   species s ON (a.species_id = s.id)
    JOIN   bio_class c ON (s.class_id = c.id)
    GROUP BY CUBE (color, tame), ROLLUP(class, species)
    ORDER BY (a.color, a.tame, c.name, s.name) DESC NULLS FIRST
    ) sc ;

63 rows affected.


sum,color,tame,species,class,grouping_bitmap,grouping_bitmap_1
37,None,None,None,None,15,1111
10,None,None,None,None,7,0111
8,None,None,None,reptile,13,1101
4,None,None,turtle,reptile,12,1100
4,None,None,lizard,reptile,12,1100
10,None,None,None,mollusk,13,1101
10,None,None,None,mollusk,5,0101
10,None,None,cuttlefish,mollusk,12,1100
10,None,None,cuttlefish,mollusk,4,0100
19,None,None,None,mammal,13,1101


[Reference](https://www.depesz.com/2015/05/24/waiting-for-9-5-support-grouping-sets-cube-and-rollup/), [reference](https://www.postgresql.org/docs/devel/static/queries-table-expressions.html)